<a href="https://colab.research.google.com/github/dvschultz/ml-art-colabs/blob/master/Stylegan2_ada_Custom_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [1]:
# mount Google Drive on the runtime
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# create a symbolic link to a working directory
!ln -s /content/gdrive/My\ Drive/Colab\ Notebooks/monixypAI /mydrive

# navigate to the working directory
%cd /mydrive

ln: failed to create symbolic link '/mydrive/monixypAI': File exists
/content/gdrive/My Drive/Colab Notebooks/monixypAI


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
!nvidia-smi

Sat Mar  6 17:35:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [6]:
import os
if os.path.isdir("/mydrive/stylegan2-ada"):
    %cd "/mydrive/stylegan2-ada"
else:
    #install script
    %cd "/mydrive/"
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/gdrive/My Drive/Colab Notebooks/monixypAI/stylegan2-ada


In [7]:
# !git config --global user.name "test"
# !git config --global user.email "test@test.com"
# !git fetch origin
# !git checkout origin/main -- train.py

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [13]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/mydrive/stylegan2-ada/datasets/downscaled.zip"

!unzip {zip_path} -d /mydrive/stylegan2-ada/datasets/monixyp/

Archive:  /mydrive/stylegan2-ada/datasets/downscaled.zip
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/1.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/10b.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/10c.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/2.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/3. Monika Polak, artery technika wИasna na pИвtnie,20x20cm, 2019.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/4. Monika Polak, ultracell, technika wИasna na pИвtnie,20x20 cm,  2019.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/4.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/5. Monika Polak, szalka (z cyklu Naczynia), akwarela na pИвtnie, 20x20cm, 2018.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/5.png  
  inflating: /mydrive/stylegan2-ada/datasets/monixyp/6. Monika Polak, Kontakt, (z cyklu Naczynia), 60x60cm, akwarela na pИвtnie, 2018.png  
  inflating: /mydrive/stylegan2-ada/datas

In [14]:
from PIL import Image
PATH = '/mydrive/stylegan2-ada/datasets/monixyp/'
SIZE = (256, 256)
for n, file in enumerate(os.listdir(PATH)):
  image = Image.open(PATH+file) \
               .convert('RGB') \
               .resize(SIZE)
  image.save(PATH+f'{n}.jpeg', 'jpeg')
  os.remove(PATH+file)

In [15]:
# os.listdir('/mydrive/stylegan2-ada/datasets/monixyp/')
# !ls

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [16]:
%cd '/mydrive/stylegan2-ada/'

#update this to the path to your image folder
dataset_path = '/mydrive/stylegan2-ada/datasets/monixyp/'
#give your dataset a name
dataset_name = 'monixyp_tfrecords'

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

/content/gdrive/My Drive/Colab Notebooks/monixypAI/stylegan2-ada
Loading images from "/mydrive/stylegan2-ada/datasets/monixyp/"
Creating dataset "./datasets/monixyp_tfrecords"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 144 images.


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [17]:
# !python train.py --help

In [18]:
# !ls

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = 'monixyp_tfrecords'
#how often should the model generate samples and a .pkl file
snapshot_count = 2
#should the images be mirrored left to right?
mirrored = False
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = 'bg'

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
# resume_from = "/content/drive/My\ Drive/colab-sg2-ada2/stylegan2-ada/results/00008-dante1024-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000160.pkl"
# resume_from = './ffhq.pkl'
resume_from = 'ffhq256'

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results \
                 --snap={snapshot_count} \
                 --cfg=11gb-gpu \
                 --data=./datasets/{dataset_name} \
                 --augpipe={augs} \
                 --mirror={mirrored} \
                 --mirrory={mirroredY} \
                 --metrics={metric_list} \
                #  --resume={resume_from} \
                #  --augpipe="bg" 

tcmalloc: large alloc 4294967296 bytes == 0x55f9d77fe000 @  0x7fcd087e2001 0x7fcd059e554f 0x7fcd05a35b58 0x7fcd05a39b17 0x7fcd05ad8203 0x55f9cfbe20e4 0x55f9cfbe1de0 0x55f9cfc566f5 0x55f9cfc50e0d 0x55f9cfbe402c 0x55f9cfc24d39 0x55f9cfc21c84 0x55f9cfbe28e9 0x55f9cfc56ade 0x55f9cfc50b0e 0x55f9cfb22e2b 0x55f9cfc531e6 0x55f9cfc50b0e 0x55f9cfb22e2b 0x55f9cfc531e6 0x55f9cfc50e0d 0x55f9cfb22e2b 0x55f9cfc531e6 0x55f9cfbe369a 0x55f9cfc51a45 0x55f9cfc50b0e 0x55f9cfc50813 0x55f9cfd1a592 0x55f9cfd1a90d 0x55f9cfd1a7b6 0x55f9cfcf2103
tcmalloc: large alloc 4294967296 bytes == 0x55fad77fe000 @  0x7fcd087e01e7 0x7fcd059e546e 0x7fcd05a35c7b 0x7fcd05a3635f 0x7fcd05ad8103 0x55f9cfbe20e4 0x55f9cfbe1de0 0x55f9cfc566f5 0x55f9cfc50b0e 0x55f9cfbe377a 0x55f9cfc5286a 0x55f9cfc50b0e 0x55f9cfbe377a 0x55f9cfc5286a 0x55f9cfc50b0e 0x55f9cfbe377a 0x55f9cfc5286a 0x55f9cfbe369a 0x55f9cfc51a45 0x55f9cfc50b0e 0x55f9cfbe377a 0x55f9cfc55e50 0x55f9cfc50b0e 0x55f9cfbe377a 0x55f9cfc5286a 0x55f9cfc50e0d 0x55f9cfbe402c 0x55f9cfc2

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.